# 実務で活かしたい人のための実践的ベイズ統計モデリングハンズオン

- ベイズ統計モデリングをビジネスに生かすにはという視点で、ハンズオンをベースに解説していきます
- 具体的な事例として、広告や口コミ評価などの効果比較を扱います
- 広告が題材ではありますが、データ数にバラツキがあったり、データ数が少ない場合の比較や予測に応用できます


In [ ]:
# ライブラリのインポート
import sys, os
import numpy as np
import pandas as pd
import scipy.stats as stats
import math
import pymc3 as pm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# プロットする図を綺麗にする
sns.set()

## ハンズオンアジェンダ
- [広告効果の単純比較](#広告効果の単純比較)
- [ベイズ推定による広告効果の比較](#ベイズ推定による広告効果の比較)
- [ベイズ的A/Bテスト](#ベイズ的A/Bテスト)

# 広告効果の単純比較

## 問題設定
- バナー広告Aとバナー広告Bの効果を比較する
- AとBのバナーは広告する商材は一緒だが、クリエイティブ（バナー広告のデザイン）が異なるものとする
- バナーAは長期間配信しているものだが、バナーBは最近新たな施策の結果として作られた新しいバナーである
- 広告効果はクリック率（CTR）の大小で比較する。つまり、バナーAとBそれぞれの**CTRを推定**する問題となる。

## モデル設定
- Clickイベントはバナー広告毎に、ある確率$\theta_{A} or \theta_{B}$で発生すると仮定
  - 訪問者それぞれのクリック有無は独立で、訪問者それぞれに特性は無いと仮定する（i.i.d:独立同分布）
- ユーザとクリック有無それぞれは独立と仮定するため、ベルヌーイ分布で表現できる

$
f(x|\theta) = \theta^{x}(1-\theta)^{1-x}
$

- バナー広告A、Bのクリック確率（$\theta_A, \theta_B$）を推定することが目的となる

## データの生成
- バナー広告がクリックされたか否かはベルヌーイ分布に従うと仮定（i.i.d）

In [ ]:
# 乱数シードの設定
np.random.seed(25) # 25

# 真のクリック率の設定（この値を推定したい）
p_a_true = 0.010 # 0.010
p_b_true = 0.015 # 0.015

# 訪問者數
N_a = 1000 # 1000
N_b = 100 # 100

# シミュレーション
click_a = stats.bernoulli(p=p_a_true).rvs(size=N_a)
click_b = stats.bernoulli(p=p_b_true).rvs(size=N_b)

# データを確認
summary = pd.DataFrame(
    {
        'access':[len(click_a), len(click_b)], 
        'click_num':[np.sum(click_a), np.sum(click_b)], 
        'freq':[np.mean(click_a), np.mean(click_b)], 
        'true_p':[p_a_true, p_b_true]
    }, 
    index=['A', 'B']
)
summary

## 最尤推定でのCTR推定
- バナーA,BのCTRを推定するために、単純に頻度を算出するケースが多い
- これは、Clickイベントをベルヌーイ分布（または、二項分布）と仮定した場合の最尤推定量と一致する
  - モデル（ベルヌーイ分布）
    - x:クリック有無{0,1}
    - $\theta$:クリック確率
  $$
  f(x|\theta) = \theta^{x}(1-\theta)^{1-x}
  $$
  - 尤度関数$L(\theta|X)$の最大化
$$
L(\theta|X)=\prod^{N}_{i=1}{f(x_i|\theta)} = \prod^{N}_{i=1}\theta^{x_i}(1-\theta)^{1-x_i} \\
\log{L(\theta|X)} = \sum^{N}_{i=1}{x_i\log{\theta}+(1-x_i)\log{(1-\theta)} } \\
\qquad = m\log{\theta} + (N-m)\log{{1-\theta}}
$$

微分して0となるパラメータ

$$
\frac{d\log{L(\theta|X)}}{d\theta} = 0 \\
\hat{\theta} = \frac{m}{N}
$$

ということで、長々と数式を展開してきたが、結局は標本平均となる。


In [ ]:
# 上記の集計ですでに計算していたので
summary

In [ ]:
# 真のクリック確率（設定値）と推定値のプロット

fig = plt.figure(figsize=(12, 5))
ax = fig.subplots(1, 1)

cs = ['#FF4500', '#0000FF', '#00F1A1']
cnt=0
for idx in summary.index:
    ax.vlines(summary.loc[idx, 'freq'], 0, 1, colors=cs[cnt%len(cs)], label='MLE_{}'.format(idx))
    ax.vlines(summary.loc[idx, 'true_p'], 0, 1, colors=cs[cnt%len(cs)], linestyle='dashed', label='True_{}'.format(idx))
    cnt+=1
ax.set_xlim((0.0, 0.05))
ax.set_xlabel('$\\theta$')
ax.legend()

In [ ]:
# 推定値の差分。正の値であれば広告Bの方がクリック率が高いと推定
print('difference between B and A : {}'.format(summary.loc['B', 'freq']-summary.loc['A', 'freq']))

## 広告効果の比較まとめ
最尤推定（頻度）で広告効果を推定した。

その結果は想定したものであっただろうか？

単純に頻度を比較するだけでは、このような問題が生じている可能性がある。

## 課題
この結果は設定した真の値(p_a_true, p_b_true)と比較してどうなっているだろうか？

- p_true, Nを変えながら結果を考察してみる

# ベイズ推定による広告効果の比較
- 最尤推定による推定結果はデータに強く依存し、特にデータ数が少ない場合にはデータに過度に適合してしまう（過適合, over-fitting）
- 差を比較したいが、その差に意味がある差なのか直感的にわかりにくい
- そこで、ベイズ推定によって広告A,Bの効果を推定し、その差を比較してみる

## ベイズ推定でのCTR推定
- ベイズの式
    - $p(\theta | x) \propto p(x | \theta)p(\theta)$
    - x : バナー広告をn回表示した結果のクリック数
    - $\theta$ : 確率分布のパラメータ（バナー広告のクリック確率）
    - 尤度$p(x | \theta)$と事前分布$p(\theta)$を設定する必要がある
- 尤度関数
    - 二項分布
    - $p(x | \theta) = \binom{n}{x}\theta^x(1-\theta)^{N-x}$
- 事前分布
    - ベータ分布
    - $p(\theta) = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}\theta^{\alpha-1}(1-\theta)^{\beta-1}$


### ベータ分布
- 二項分布の共役事前分布という性質の他、扱いやすい特徴を持っている
  - 値域が0~1の範囲なので確率を扱うのに都合が良い
  - αとβの値によって様々な形状の分布を表現できる

ベータ分布の形状を確認してみる

In [ ]:
params = [0.1, 1, 2, 10] # alpha, betaの例

x = np.linspace(0, 1, 100) # x軸の設定

fig = plt.figure(figsize=(13, 10))
ax = fig.subplots(len(params), len(params), sharex=True, sharey=True)
cnt=0
for i in range(len(params)):
    for j in range(len(params)):
        # パラメータalphaとbetaを設定
        a = params[i]
        b = params[j]
        # ベータ分布の確率密度を計算
        y = stats.beta(a, b).pdf(x)
        # plot
        ax[i, j].plot(x, y)
        ax[i, j].plot(0, 0, label="$\\alpha$ = {:3.2f}\n$\\beta$ = {:3.2f}".format(a, b), alpha=0)
        ax[i, j].legend()
        if i == (len(params)-1):
            ax[i,j].set_xlabel('$\\theta$')
        if j == 0:
            ax[i,j].set_ylabel('$p(\\theta)$')

### ベイズ推定によるパラメータ推定
- MCMCアルゴリズムの実行はPythonのライブラリであるPyMC3を利用する

In [ ]:
# データの整理
p_trues = [p_a_true, p_b_true]
clicks = [click_a, click_b]
n_imps = summary['access'].values
n_clicks = summary['click_num'].values
banners = summary.index.values
banner_idx = np.arange(0, len(banners))

In [ ]:
# MCMCアルゴリズムの実行
n_sample = 1000

with pm.Model() as model_banner_effect:
    # prior
    theta = pm.Beta('theta', alpha=1, beta=1, shape=len(banners))
    # Likelihood
    y = pm.Binomial('y', n=n_imps[banner_idx], p=theta[banner_idx], observed=n_clicks[banner_idx])
    # sample
    trace_banner = pm.sample(n_sample, chains=3)

pm.traceplot(trace_banner)

In [ ]:
# 事後分布の確認
fig = plt.figure(figsize=(12, 3*2))
ax = fig.subplots(2, 1, sharex=True)

bins = np.linspace(0, 0.1, 100)
for i in banner_idx:
    ret = sns.distplot(trace_banner['theta'][:,i], bins=bins, ax=ax[i], label='posterior')
    ax[i].vlines(p_trues[i], ret.get_ylim()[0], ret.get_ylim()[1], color='red', label='p_true')
    ax[i].vlines(clicks[i].mean(), ret.get_ylim()[0], ret.get_ylim()[1], color='blue', label='p_MLE')
    ax[i].set_ylabel('Banner {}'.format(banners[i]))
    ax[i].legend()

## CTRの差を確認
- 事後分布を並べただけでは二つの広告のCTRに差があるのか見えにくい
- 差の分布を確認する
  - 関数の差をとる必要があるが、サンプルした結果があるため、サンプル間の差の分布を見れば良い

In [ ]:
theta_diff = trace_banner['theta'][:, 1] - trace_banner['theta'][:, 0]

fig = plt.figure(figsize=(12, 5))
ax = fig.subplots(1, 1)

ret = sns.distplot(theta_diff, ax=ax)
ax.vlines((p_trues[1] - p_trues[0]), 0, ret.get_ylim()[1], color='red', label='diff_true')
ax.vlines(0.0, 0, ret.get_ylim()[1], linestyle='dashed')
ax.legend()
ax.set_title('Banner B - A')

print('Banner B is WORTH than A ;', (theta_diff < 0).mean())
print('Banner B is BETTER than A ;', (theta_diff > 0).mean())
#pm.plot_posterior(theta_diff)

## ベイズ推定による広告効果比較のまとめ

ここまでで、バナーA,Bの効果を推定し、差を評価してみた。

最尤指定量では、単純にバナーAの方が効果が高いと出てしまったが、データ量を考慮して事後分布を推定したところ、バナーBの方が効果が高い可能性がありそうなことがわかった。

しかし、手元にあるデータでは、その差はほとんど見えてこなかった。


## 課題
- このような結果が出た場合にどのような意思決定をするべきだろうか？
  - 各バナーの事後分布の形状（広がり）に注目するとわかりやすい

# ベイズ的A/Bテスト
- ここまでは、バナー広告のCTRを推定をすることで、どのバナーが効果が高いのかを分析してきた
- ここまでの結果を持って、どちらのバナー広告に力を入れるかと言う意思決定ができるかもしれない
- ここからは、A/Bテストの結果の分析について一歩踏み込んだ分析を行う

## CTR増加量を求める
- ここまでは、バナーA,Bでどちらが有効かについて分析を行った
- 「どちらのバナーが有効か」についてはわかったが、配信を変更することで「どのくらい増加するのか？」については、明確な回答を示せていない（事後分布で結果を示しているため）
- CTRがいくら増加するのかを見積もってみる

### よくあるやり方
バナーAからBへ変更した結果、CTRの増加量がどのくらいになりそうかとの課題について、通常は以下のような増加量を算出するであろう

$
rift_{AB} = \frac{\hat{p}_B - \hat{p}_A}{\hat{p}_A}
$

ここで、$\hat{p}$はバナー広告AまたはBのクリック確率の（点）推定値。
最尤推定での推定結果や、ベイズ推定後の事後分布のMAPや期待値などの点推定結果。

In [ ]:
# 事後分布の期待値（平均）を使って増加量を計算してみる
print('predicted click rate (banner A) : {}'.format(trace_banner['theta'][:, 0].mean()))
print('predicted click rate (banner B) : {}'.format(trace_banner['theta'][:, 1].mean()))
rift_point = (trace_banner['theta'][:, 1].mean() - trace_banner['theta'][:, 0].mean())/(trace_banner['theta'][:, 0].mean())
print('rift value (A -> B, point_estimation) : {}'.format(rift_point))

しかしこのやり方は深刻な問題を引き起こす場合がある。
$\hat{p}$が小さい場合、上記の値はすごく大きな値になることがある。  
（効果試算の結果300%向上みたいな結果、素直に信じますか？）  
これは、バナー広告を入れ替えた後の効果測定の際にも同じことが言える。

推定値の不確実性(事後分布のばらつき)を含めて増加量を見積もる必要がある。

### 事後分布を利用したやり方
事後分布を利用した増加割合を計算するには、確率密度関数の比を取る必要がある。  
しかし、事後分布の推定をサンプルベースの方法で行なっているため、サンプル間の増加量の分布を計算すれば良い。

In [ ]:
rift_posterior = (trace_banner['theta'][:, 1] - trace_banner['theta'][:, 0])/trace_banner['theta'][:, 0]

fig = plt.figure(figsize=(12, 5))
ax = fig.subplots(1, 1)

ret = sns.distplot(rift_posterior, ax=ax)
ax.set_xlabel('rift value')

print('rift value (A -> B, over 20% up):{}'.format((rift_posterior>0.2).mean()))
print('rift value (A -> B, over 50% up):{}'.format((rift_posterior>0.5).mean()))

20%向上する確率、50%向上する確率が出せた。

信用区間を使って、増加量を見積もる（90%信用区間）。

In [ ]:
alpha = 0.1
rift_posterior_hpd = pm.hpd(rift_posterior, alpha=alpha)
print('{}% HPD : {} ~ {}'.format((1.0-alpha)*100, rift_posterior_hpd[0], rift_posterior_hpd[1]))

### CTR増加量を求めるのまとめ
事後分布を利用し、不確実性を含めてバナーAからBに移行した場合のCTRの増加量を推定した。
推定結果は、やはり分布で得られた。

実際のシーンでは、分布で結果を提示するのではなく、点で定時する必要がある場合があるかもしれない。
そのような時には、平均（期待値）や中央値、パーセンタイルなど分布を代表する値を示す。

しかし、CTRのようなものは本質的に「真の値」なんてものは存在しない。
そのため、必ず点推定の結果と実体はブレる。
このことを認識してもらう努力は必要だと感じる。

### 課題
増加量を説明するための指標を算出し、それを上記の増加量の分布に追記してみる。

## 期待収益を求める
ここまでは、バナー広告のKPIとしてよく用いられるCTRについての考察をしてきた。  
次は、広告をClickしてもらったことで得られる期待収益について分析してみる。

### 問題設定
- バナー広告A,Bについて、それぞれのクリック率とその後の商品ページのコンバージョン率とその期待収益を合わせてA,Bの比較をする

### データ生成
- バナー広告の効果試算の結果から、さらにデータを収集したと仮定
- 合わせて商品購入ページにおけるデータも収集したと仮定

In [ ]:
# クリックデータの生成

# 乱数シードの設定
np.random.seed(25) # 25

# 真のクリック率の設定
p_a_true = 0.010 # 0.010
p_b_true = 0.015 # 0.015

# 訪問者數
N_a = 10000 # 10000
N_b = 15000 # 15000

# シミュレーション
click_a = stats.bernoulli(p=p_a_true).rvs(size=N_a)
click_b = stats.bernoulli(p=p_b_true).rvs(size=N_b)

summary = pd.DataFrame(
    {
        'access':[len(click_a), len(click_b)], 
        'click_num':[np.sum(click_a), np.sum(click_b)], 
        'freq':[np.mean(click_a), np.mean(click_b)], 
        'true_p':[p_a_true, p_b_true]
    }, 
    index=['A', 'B']
)
summary

In [ ]:
# 商品ページにおける成約数の設定

# AのCV設定
N_A = 99
N_A_1 = 2
N_A_2 = 8
N_A_3 = 20
N_A_0 = N_A - (N_A_1 + N_A_2 + N_A_3)
obs_A = np.array([N_A_1, N_A_2, N_A_3, N_A_0])

# BのCV設定
N_B = 220
N_B_1 = 2
N_B_2 = 5
N_B_3 = 20
N_B_0 = N_B - (N_B_1 + N_B_2 + N_B_3)
obs_B = np.array([N_B_1, N_B_2, N_B_3, N_B_0])

# 各商品の売り上げ額の設定
price = [7900, 4900, 2500, 0]

### 商品成約率の計算
- 商品は複数あるため、多項分布を考える
  - 各商品の成約率は独立ではない（商品１の成約率が高ければ、他は下がる）
- 事前分布には、多項分布の共役事前分布であるディリクレ分布を利用
- MCMCの練習も兼ねてPyMCを利用した数値解を求めているが、共役事前分布のため、ディリクレ分布から直接サンプルしても良い（というかそっちの方が速い）

In [ ]:
n_sample = 1000
prior_parameter = np.array([1, 1, 1, 1])
with pm.Model() as model:
    mu = pm.Dirichlet('mu', a=prior_parameter)
    cv = pm.Multinomial('cv', n=N_A, p=mu, observed=obs_A)
    trace_A = pm.sample(n_sample, chains=3)

prior_parameter = np.array([1, 1, 1, 1])
with pm.Model() as model:
    mu = pm.Dirichlet('mu', a=prior_parameter)
    cv = pm.Multinomial('cv', n=N_B, p=mu, observed=obs_B)
    trace_B = pm.sample(n_sample, chains=3)

In [ ]:
fig = plt.figure(figsize=(12, 3*2))
ax = fig.subplots(2, 1, sharex=True)

for i in np.arange(trace_A['mu'].shape[1]):
    l = 'item_{}'.format(i+1)
    if i == trace_A['mu'].shape[1]-1:
        l = 'leave'
    sns.distplot(trace_A['mu'][:,i], ax=ax[0], label=l)
ax[0].set_ylabel('from banner A')
ax[0].legend()
for i in np.arange(trace_B['mu'].shape[1]):
    l = 'item_{}'.format(i+1)
    if i == trace_B['mu'].shape[1]-1:
        l = 'leave'
    sns.distplot(trace_B['mu'][:,i], ax=ax[1], label=l)
ax[1].set_ylabel('from banner B')
ax[1].legend()
ax[1].set_xlabel('CV Rate')

### 期待収益の計算

In [ ]:
def expected_revenue(P, r=[7900, 4900, 2500, 0]):
    samples, items = P.shape
    reward = 0.0
    for i in np.arange(items):
        reward+=r[i]*P[:,i]
    return reward

r_A = price
posterior_expected_r_A = expected_revenue(trace_A['mu'], r=r_A)

r_B = price
posterior_expected_r_B = expected_revenue(trace_B['mu'], r=r_B)

fig = plt.figure(figsize=(8, 3))
ax = fig.subplots(1, 1, sharex=True)
sns.distplot(posterior_expected_r_A, label='A', ax=ax)
sns.distplot(posterior_expected_r_B, label='B', ax=ax)
ax.set_xlabel('Expected Reward')
plt.legend()

In [ ]:
# 収益差の事後分布
reward_diff = posterior_expected_r_A - posterior_expected_r_B

print('probability A > B :', (reward_diff>0).mean())

fig = plt.figure(figsize=(8, 3))
ax = fig.subplots(1, 1)
sns.distplot(reward_diff, ax=ax)
ax.set_xlabel('Difference of Reward')


### クリック確率の推定

In [ ]:
# CTRの推定
p_trues = [p_a_true, p_b_true]
clicks = [click_a, click_b]
n_imps = summary['access'].values
n_clicks = summary['click_num'].values
banners = summary.index.values
banner_idx = np.arange(0, len(banners))

n_sample = 1000
with pm.Model() as model_banner_effect:
    # prior
    theta = pm.Beta('theta', alpha=1, beta=1, shape=len(banners))
    # Likelihood
    y = pm.Binomial('y', n=n_imps[banner_idx], p=theta[banner_idx], observed=n_clicks[banner_idx])
    # sample
    trace_banner = pm.sample(n_sample, chains=3)

fig = plt.figure(figsize=(12, 3*2))
ax = fig.subplots(2, 1, sharex=True)
bins = np.linspace(0, 0.04, 100)
for i in banner_idx:
    ret = sns.distplot(trace_banner['theta'][:,i], bins=bins, ax=ax[i], label='posterior')
    ax[i].vlines(p_trues[i], ret.get_ylim()[0], ret.get_ylim()[1], color='red', label='p_true')
    ax[i].vlines(clicks[i].mean(), ret.get_ylim()[0], ret.get_ylim()[1], color='blue', label='p_MLE')
    ax[i].set_ylabel('Banner {}'.format(banners[i]))
    ax[i].legend()
ax[1].set_xlabel('Click Rate')

### クリック確率と期待収益を合わせる

In [ ]:
access_rev_A = posterior_expected_r_A * trace_banner['theta'][:,0]
access_rev_B = posterior_expected_r_B * trace_banner['theta'][:,1]

fig = plt.figure(figsize=(8, 4))
ax = fig.subplots(1, 1)

bins=np.linspace(0, 50, 51)
ax.hist(access_rev_A, label='A', bins=bins, alpha=0.5)
ax.hist(access_rev_B, label='B', bins=bins, alpha=0.5)
ax.legend()
ax.set_xlabel('Expected Reward')

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax = fig.subplots(1, 1)

ax.hist(access_rev_A-access_rev_B)

### 期待収益の算出のまとめ
- 期待収益を算出するために、各商品の成約率を推定した
- 商品は複数あるため、多項分布とディリクレ分布を使って推定を行った


### 課題
- CV設定やクリック設定を色々変えてどのように変わるかを確認してみる